<a href="https://colab.research.google.com/github/AntonioCuba123/skills-introduction-to-github/blob/main/Copia_de_Optimizaci%C3%B3n_de_Producci%C3%B3n_de_CIFRUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este es un escenario más cercano a la planificación real, pero también introduce un problema de optimización complejo. Vamos a dividir esto en partes:

Parte 1: Generar Datos Sintéticos: Crear datos históricos para Cifrut Naranja en varias ubicaciones que representen una "red de distribución" (vamos a simular algunos CEDIS y bodegas clave).

Parte 2: Pronóstico para el Plan de Producción (+5%): Pronosticar la demanda total (agregada en toda la red simulada) de Cifrut Naranja para el próximo mes y añadir un 5% de buffer.

Paete 3: Pronóstico para la Optimización de Producción (Cifrut Naranja): Pronosticar la demanda de Cifrut Naranja para las próximas 2 semanas, pero esta vez por ubicación (CEDIS y bodegas principales).

Parte 4: Explicar el Problema de Optimización: Describir qué se necesita para resolver la parte de "cantidad óptima a producir y cuándo" (que es un problema de optimización de la cadena de suministro) y por qué no podemos codificar una solución completa aquí (requiere datos y herramientas especializadas).

Parte 1: Generación de Datos Sintéticos (Cifrut Naranja en la Red de Distribución)

Vamos a simular un año de datos diarios para Cifrut Naranja en una red que podría incluir algunos CEDIS importantes y un conjunto de bodegas "principales" distribuidas geográficamente.

In [ ]:
# @title Generación de Datos Sintéticos para Cifrut Naranja en la Red (Ejecutar esta celda)

import pandas as pd
import numpy as np
from datetime import date, timedelta
import random

# --- Parámetros de la Red de Distribución Simulada ---
# Vamos a simular algunos CEDIS y un conjunto de bodegas distribuidas en ciudades clave
cedis_ubicaciones = ['CEDIS_Lima', 'CEDIS_Piura', 'CEDIS_Trujillo', 'CEDIS_Arequipa', 'CEDIS_Chiclayo']
num_bodegas_por_ciudad = 40 # 40 bodegas principales en cada una de estas 5 ciudades
bodegas_ubicaciones = []
for ciudad in ['Lima', 'Trujillo', 'Piura', 'Arequipa', 'Chiclayo']:
    bodegas_ubicaciones.extend([f'Bodega_{ciudad}_{i+1}' for i in range(num_bodegas_por_ciudad)])

ubicaciones_red = cedis_ubicaciones + bodegas_ubicaciones # Total: 5 CEDIS + 200 Bodegas
sku = 'Cifrut Naranja' # El SKU específico

dias_historicos = 365 # Un año de datos históricos

# Definimos la fecha de fin como hoy (o una fecha reciente)
fecha_fin = date.today()
fecha_inicio = fecha_fin - timedelta(days=dias_historicos - 1)

# --- Parámetros para simular ventas de Cifrut Naranja ---
# Bases de ventas promedio por día (ajustar para simular volúmenes reales)
base_ventas_cedis = 2500 # Un CEDIS vende ~2500 unidades/día de este SKU
base_ventas_bodega = 60   # Una bodega principal vende ~60 unidades/día de este SKU

# Variación aleatoria típica (ruido)
ruido_cedis = 400
ruido_bodega = 15

# Factores para simular ventas más altas/bajas en ciertos días (similar al ejemplo anterior)
factor_dia_semana = {
    0: 1.0,  # Lunes
    1: 1.05, # Martes
    2: 1.1,  # Miércoles
    3: 1.15, # Jueves
    4: 1.25, # Viernes
    5: 1.3,  # Sábado
    6: 1.2   # Domingo
}

# Simular variabilidad entre bodegas
np.random.seed(43) # Nueva semilla para este escenario
factores_ubicacion = {loc: np.random.uniform(0.7, 1.3) for loc in bodegas_ubicaciones} # Factores para bodegas
# Los CEDIS pueden tener menos variabilidad aleatoria en su base, o factores específicos si se conocen
for cedis in cedis_ubicaciones:
     factores_ubicacion[cedis] = np.random.uniform(0.9, 1.1) # Menos variabilidad para CEDIS base

# Simular estacionalidad anual (ej: ventas más altas en verano - Ene/Feb/Mar en Perú)
def factor_estacional_anual(fecha):
    dia_del_año = fecha.timetuple().tm_yday
    dia_pico_verano = 45 # Pico en Febrero
    # Onda sinusoidal con un rango del 30% alrededor de la base (1 +/- 0.15)
    return 1 + 0.15 * np.cos((dia_del_año - dia_pico_verano) * 2 * np.pi / 365)


# --- Proceso de Generación de Datos ---
datos = []
fechas = pd.date_range(start=fecha_inicio, periods=dias_historicos, freq='D')

for ubicacion in ubicaciones_red:
    tipo_ubicacion = 'CEDIS' if 'CEDIS' in ubicacion else 'Bodega'

    if tipo_ubicacion == 'CEDIS':
        base_v = base_ventas_cedis
        ruido_v = ruido_cedis
        factor_u = factores_ubicacion.get(ubicacion, 1.0) # Usa el factor si existe, sino 1.0
    else: # Bodega
        base_v = base_ventas_bodega
        ruido_v = ruido_bodega
        factor_u = factores_ubicacion.get(ubicacion, 1.0) # Usa el factor si existe, sino 1.0

    for fecha in fechas:
        # Calcular ventas base con factor de ubicación
        ventas_base = base_v * factor_u

        # Añadir ruido diario aleatorio
        ruido = np.random.normal(0, ruido_v)

        # Aplicar factor del día de la semana
        factor_ds = factor_dia_semana[fecha.weekday()]

        # Aplicar factor estacional anual
        factor_est = factor_estacional_anual(fecha)

        # Calcular ventas simuladas
        ventas_simuladas = (ventas_base + ruido) * factor_ds * factor_est

        # Asegurarse de que las ventas no sean negativas y sean números enteros
        ventas_finales = max(0, int(round(ventas_simuladas)))

        # Añadir algo de escasez aleatoria o días sin venta para realismo en bodegas pequeñas
        if tipo_ubicacion == 'Bodega' and random.random() < 0.03: # 3% de probabilidad de 0 ventas en una bodega
             ventas_finales = 0

        datos.append([fecha, ubicacion, tipo_ubicacion, sku, ventas_finales])

# Crear el DataFrame
df_ventas_cifrut = pd.DataFrame(datos, columns=['Fecha', 'Ubicacion', 'TipoUbicacion', 'SKU', 'Ventas'])

# Mostrar las primeras filas y información del DataFrame
print("--- Datos Históricos Sintéticos Generados (Cifrut Naranja) ---")
print(df_ventas_cifrut.head())
print("\nInformación del DataFrame:")
df_ventas_cifrut.info()
print("\nEstadísticas descriptivas de las ventas por TipoUbicacion:")
print(df_ventas_cifrut.groupby('TipoUbicacion')['Ventas'].describe())
print("\nEstadísticas descriptivas de las ventas totales:")
print(df_ventas_cifrut['Ventas'].describe())


# Guardar el DataFrame en un archivo CSV
nombre_archivo_historico_cifrut = "ventas_historicas_cifrut_red.csv"
df_ventas_cifrut.to_csv(nombre_archivo_historico_cifrut, index=False)
print(f"\nDatos históricos de Cifrut Naranja guardados en '{nombre_archivo_historico_cifrut}'")

Creamos datos diarios para un año.

Definimos un conjunto de ubicaciones que representan la "red": 5 CEDIS y 200 bodegas principales distribuidas en 5 ciudades.

Simulamos ventas solo para el SKU 'Cifrut Naranja'.
Ajustamos las bases de ventas y el ruido para simular volúmenes más altos típicos de Cifrut Naranja y diferenciamos claramente CEDIS de Bodegas.

Mantenemos la simulación de factores por día de la semana y estacionalidad anual.
**texto en negrita**
Guardamos los datos en un archivo CSV específico para este SKU y red.

Parte 2: Pronóstico para el Plan de Producción (Próximo Mes +5%)
Para el plan de producción, usualmente se necesita la demanda total agregada en un período. Pronosticaremos la demanda total de Cifrut Naranja en toda la red simulada para el próximo mes y añadiremos el buffer del 5%.

In [ ]:
# @title Pronóstico para Plan de Producción (Próximo Mes +5%) (Ejecutar esta celda)

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt # Para visualización opcional

# 1. Cargar los datos históricos de Cifrut Naranja
nombre_archivo_historico_cifrut = "ventas_historicas_cifrut_red.csv"
df_ventas_cifrut = pd.read_csv(nombre_archivo_historico_cifrut)

# Convertir la columna de fecha a formato datetime
df_ventas_cifrut['Fecha'] = pd.to_datetime(df_ventas_cifrut['Fecha'])

# 2. Agregar los datos históricos para obtener la Venta Diaria Total de la Red
# Agrupamos por fecha y sumamos las ventas de todas las ubicaciones para ese día
df_total_diario = df_ventas_cifrut.groupby('Fecha')['Ventas'].sum().reset_index()

# Preparar los datos agregados para Prophet
df_prophet_total = df_total_diario.rename(columns={'Fecha': 'ds', 'Ventas': 'y'})

# 3. Crear y entrenar el modelo Prophet para la Demanda Total
print("Entrenando modelo Prophet para la demanda total de la red...")
model_total = Prophet(
    seasonality_mode='additive',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False
)

model_total.fit(df_prophet_total)

# 4. Crear el DataFrame futuro para el próximo mes (aprox. 30 días)
dias_proximo_mes = 30
future_total = model_total.make_future_dataframe(periods=dias_proximo_mes, include_history=False)

# 5. Hacer la predicción de la demanda total
forecast_total = model_total.predict(future_total)

# 6. Calcular el Plan de Producción (+5% buffer)
# Seleccionamos las columnas de fecha y la predicción puntual ('yhat')
plan_produccion_diario = forecast_total[['ds', 'yhat']].copy()

# Añadimos el buffer del 5% a la predicción
plan_produccion_diario['produccion_estimada'] = plan_produccion_diario['yhat'] * 1.05

# Asegurarse de que la producción estimada sea entera y no negativa
plan_produccion_diario['produccion_estimada'] = plan_produccion_diario['produccion_estimada'].apply(lambda x: max(0, round(x))).astype(int)
plan_produccion_diario['yhat'] = plan_produccion_diario['yhat'].apply(lambda x: max(0, round(x))).astype(int) # Redondear la predicción original también

# 7. Mostrar el Plan de Producción Diario Estimado para el Próximo Mes
print("\n--- Plan de Producción Diario Estimado para el Próximo Mes (Cifrut Naranja, Red Total) ---")
print("Incluye Pronóstico Puntual (yhat) y Estimado con +5% Buffer (produccion_estimada)")
print(plan_produccion_diario.to_string()) # Usar to_string() para mostrar todas las filas

# 8. Calcular el total o promedio mensual para el resumen del plan
total_proximo_mes_estimado = plan_produccion_diario['produccion_estimada'].sum()
promedio_diario_proximo_mes_estimado = plan_produccion_diario['produccion_estimada'].mean()

print(f"\nTotal Estimado a Producir Próximo Mes (+5% Buffer): {total_proximo_mes_estimado} unidades")
print(f"Promedio Diario Estimado a Producir Próximo Mes (+5% Buffer): {promedio_diario_proximo_mes_estimado:.2f} unidades")

# 9. (Opcional) Visualizar el pronóstico total
# fig_total = model_total.plot(forecast_total)
# plt.title('Pronóstico de Demanda Total de Cifrut Naranja (Red)')
# plt.xlabel('Fecha')
# plt.ylabel('Demanda Estimada')
# plt.show()

Cargamos los datos históricos de Cifrut Naranja y usamos .groupby('Fecha')['Ventas'].sum() para sumar las ventas de todas las ubicaciones en cada fecha. Esto nos da la demanda diaria total de la red.

Preparar para Prophet: Transformamos el DataFrame agregado al formato ds, y requerido por Prophet.

Entrenamos un único modelo Prophet con la serie de tiempo de la demanda diaria total. Este modelo aprende los patrones de la demanda agregada de la red.

Le pedimos al modelo que prediga los valores para los próximos 30 días.
Tomamos la predicción puntual (yhat) y la multiplicamos por 1.05 para añadir el buffer del 5%. Redondeamos a entero.

Presentamos el pronóstico diario con el buffer para cada uno de los próximos 30 días. También calculamos y mostramos el total mensual y el promedio diario para el resumen.

Parte 3: Pronóstico para la Optimización de Producción (Próximas 2 Semanas por Ubicación)

La pregunta de la cantidad óptima a producir en la planta X requiere saber la demanda en cada punto de la red para poder planificar el envío y la logística. Por lo tanto, necesitamos pronosticar la demanda de Cifrut Naranja para las próximas 2 semanas, pero esta vez por ubicación individual.

In [ ]:
# @title Pronóstico para Input de Optimización (Próximas 2 Semanas, Por Ubicación) (Ejecutar esta celda)

import pandas as pd
from prophet import Prophet
# No necesitamos matplotlib aquí a menos que queramos ver gráficos individuales por ubicación

# 1. Cargar los datos históricos de Cifrut Naranja
nombre_archivo_historico_cifrut = "ventas_historicas_cifrut_red.csv"
df_ventas_cifrut = pd.read_csv(nombre_archivo_historico_cifrut)

# Convertir la columna de fecha a formato datetime
df_ventas_cifrut['Fecha'] = pd.to_datetime(df_ventas_cifrut['Fecha'])

# Preparar una lista para almacenar los resultados de los pronósticos por ubicación
resultados_pronostico_ubicacion = []

# 2. Iterar a través de cada ubicación única en la red
# Agrupamos los datos por 'Ubicacion' para obtener la serie de tiempo de ventas de cada lugar
grupos_ubicacion = df_ventas_cifrut.groupby('Ubicacion')

print(f"\nIniciando pronóstico por ubicación para {len(grupos_ubicacion)} ubicaciones...")

# Iteramos sobre cada grupo (cada ubicación)
for ubicacion, grupo_df in grupos_ubicacion:
    # print(f"Pronosticando para: {ubicacion}...") # Descomentar para ver el progreso por ubicación

    # Preparar los datos para Prophet (columnas 'ds' y 'y')
    df_prophet_ubicacion = grupo_df[['Fecha', 'Ventas']].rename(columns={'Fecha': 'ds', 'Ventas': 'y'})

    # Asegurarse de tener suficientes datos
    if len(df_prophet_ubicacion) < 2:
        # print(f"Saltando {ubicacion} - Insuficientes puntos de datos ({len(df_prophet_ubicacion)}).")
        continue

    # 3. Crear y entrenar el modelo Prophet para esta Ubicación
    # Entrenamos un modelo SEPARADO para cada ubicación
    model_ubicacion = Prophet(
        seasonality_mode='additive',
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )

    try:
        model_ubicacion.fit(df_prophet_ubicacion)

        # 4. Crear el DataFrame futuro para las próximas 2 semanas (14 días)
        dias_proximas_dos_semanas = 14
        future_ubicacion = model_ubicacion.make_future_dataframe(periods=dias_proximas_dos_semanas, include_history=False)

        # 5. Hacer la predicción para esta Ubicación
        forecast_ubicacion = model_ubicacion.predict(future_ubicacion)

        # 6. Seleccionar las columnas relevantes y añadir la Ubicación
        # 'yhat' es la predicción puntual de la demanda diaria para esta ubicación
        forecast_subset_ubicacion = forecast_ubicacion[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
        forecast_subset_ubicacion['Ubicacion'] = ubicacion
        forecast_subset_ubicacion['SKU'] = sku # Añadimos el SKU (Cifrut Naranja)

        # Asegurarse de que la predicción sea no negativa y redondear a entero
        for col in ['yhat', 'yhat_lower', 'yhat_upper']:
             forecast_subset_ubicacion[col] = forecast_subset_ubicacion[col].apply(lambda x: max(0, round(x))).astype(int)


        # Añadir al resultado global
        resultados_pronostico_ubicacion.append(forecast_subset_ubicacion)

    except Exception as e:
        print(f"Error al pronosticar para {ubicacion}: {e}")
        continue


# 7. Combinar todos los resultados de pronóstico por ubicación
if resultados_pronostico_ubicacion:
    df_pronostico_ubicacion_completo = pd.concat(resultados_pronostico_ubicacion, ignore_index=True)

    # Reordenar columnas
    df_pronostico_ubicacion_completo = df_pronostico_ubicacion_completo[['ds', 'Ubicacion', 'SKU', 'yhat', 'yhat_lower', 'yhat_upper']]

    print("\n--- Pronóstico de Demanda Diaria por Ubicación (Cifrut Naranja, Próximas 2 Semanas) ---")
    # Mostrar los resultados ordenados
    print(df_pronostico_ubicacion_completo.sort_values(by=['ds', 'Ubicacion']).to_string())

    # Opcional: Guardar este pronóstico detallado en un CSV
    nombre_archivo_pronostico_ubicacion = "pronostico_demanda_cifrut_ubicacion_14dias.csv"
    df_pronostico_ubicacion_completo.to_csv(nombre_archivo_pronostico_ubicacion, index=False)
    print(f"\nPronóstico detallado por ubicación guardado en '{nombre_archivo_pronostico_ubicacion}'")

else:
    print("\nNo se pudieron generar pronósticos por ubicación.")

Cargamos los mismos datos históricos de Cifrut Naranja.
Agrupar por Ubicación: Usamos .groupby('Ubicacion') para dividir el DataFrame grande en DataFrames más pequeños, uno por cada ubicación en nuestra red simulada.

Bucle y Entrenamiento Individual: Iteramos a través de cada grupo (cada ubicación). Dentro del bucle, entrenamos un modelo Prophet específico para la serie de ventas de Cifrut Naranja en esa única ubicación.

Pronosticar 2 Semanas: Para cada ubicación, creamos un DataFrame futuro de 14 días y usamos su modelo entrenado individualmente para predecir la demanda.
Recopilar Resultados: Recopilamos los pronósticos individuales (fecha, ubicación, SKU, predicción puntual y intervalo de confianza) en una lista y luego los combinamos en un único DataFrame.

Mostrar Resultados: Imprimimos este DataFrame, que muestra la demanda diaria pronosticada para cada una de las 205 ubicaciones (5 CEDIS + 200 Bodegas) para cada uno de los próximos 14 días.